In [1]:
# linear model module
#import modules

import numpy as np
import pandas as pd
import scipy
from tabulate import tabulate
from scipy import stats


class ols():

    #init variables

    def __init__(self, dataset, dependent, regressors, cons = True, fixed_effects = [],
    method = 'standard', cluster = []):
        self.dataset = dataset
        self.dependent = dependent
        self.reggressors = regressors
        self.cons = cons
        self.fixed_effects = fixed_effects
        self.method = method
        self.cluster = cluster
        self.cons_array = []
        if self.cons is True:
            # i need it to concatenate variables
            self.cons_array = ['cons']
        else:
            self.cons_array = []
        #get element zero from dependent vector
        self.dep_var = self.dependent[0]


    # prepare de dataset

    def prep_data(self):
        # cerate column for constant if constant is required in the model
        if self.cons is True:
            self.dataset[self.cons_array[0]] = np.ones(len(self.dataset))
        # retrive keys for regressors
        regressors = self.reggressors + self.cons_array
        # create a sub sample of all regressors and dependent
        if len(self.fixed_effects) != 0:
            sub_sample = self.dataset[self.dependent + regressors +
                                              self.fixed_effects].dropna()
            fe_dummies = pd.get_dummies(sub_sample[self.fixed_effects], drop_first=True)
            sub_sample = pd.concat([sub_sample.drop(self.fixed_effects, axis=1),
                                    fe_dummies], axis=1)

        else:
            sub_sample = self.dataset[self.dependent + regressors +
                                              self.fixed_effects].dropna()

        return {'sample': self.dataset, 'sub_sample': sub_sample}

    def coefficients(self):
        X = self.prep_data().get('sub_sample').drop(self.dependent, axis = 1)
        Y = self.prep_data().get('sub_sample')[self.dep_var]
        return np.linalg.inv(X.T@X)@(X.T@Y).to_numpy().reshape((len(X.columns),1))

    def fitted(self):
        return self.prep_data().get('sub_sample').drop(self.dependent, axis = 1)\
               @self.coefficients()

    def residuals(self):
        Y = self.prep_data().get('sub_sample')[self.dep_var].to_numpy()
        Y= Y.reshape((len(self.prep_data().get('sub_sample')),1))
        return Y - self.fitted()

    def AVAR(self):
        X = self.prep_data().get('sub_sample').drop(self.dependent, axis = 1)
        XX_inv = np.linalg.inv(X.to_numpy().T @ X.to_numpy())
        if self.method == 'standard':
            ssr = self.residuals().T@self.residuals()
            dfg = 1/ (len(X) -len(X.columns) )

            sigma_hat = ssr * dfg
            return XX_inv* sigma_hat.values

        elif self.method ==  'heter':
            e_square = (self.residuals()**2).to_numpy().reshape((1, len(self.residuals())))[0]
            e_diag = np.diag(e_square)
            return (XX_inv @ (X.to_numpy().T @ e_diag @ X.to_numpy()) @ XX_inv)

            #reference https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors
            # compare with wooldridge examples

        elif self.method == 'cluster':
            #get the sample dataset
            sample = self.prep_data().get('sample')
            # be careful that cluster var is not repeated when it is also usead
            # as fixed effect ???
            keys = self.dependent + self.reggressors + self.cons_array +\
                 self.cluster + [i for i in self.fixed_effects
            if i not in self.dependent + self.reggressors + self.cons_array + self.cluster ]

            #select the subsample from keys and dropna (like stata)
            sample  = sample[keys].dropna()
            #generate dummies from fixed effects
            dummies = pd.get_dummies(sample[self.fixed_effects], drop_first=True)
            #nsert dummies in sample dataset
            sample = pd.concat([sample, dummies], axis = 1)
            #attach residuals column
            sample['residuals'] = self.residuals()
            #take unique clusters
            clusters  = list(set(sample[self.cluster].values.reshape((1, len(sample[self.cluster])))[0]))
            #set self cluster var as index to iterate across rows in the same cluster
            sample = sample.set_index(self.cluster)
            # for loop to iterate across clusters
            shape_sigma = len(self.reggressors) + 1 + len(dummies.columns)
            # initialize a zero sigma matrix. will be the sum of Bs for each cluster
            sigma_matrix = np.zeros((shape_sigma,shape_sigma))
            for c in (clusters):
                # select the subsample for cluster c
                c_sample = sample.loc[c]
                # initialize eps as the vector of residuals of cluster c
                eps_c = c_sample['residuals'].to_numpy().reshape((len(c_sample),1))
                # inirialize the vector of regressors of cluster c
                X_c = c_sample[[i for i in c_sample.columns
                                if i not in self.dependent + self.fixed_effects + self.cluster
                                +['residuals']]].values
                # calculate Bs for each cluster and sum the zero matrix
                sigma_matrix += X_c.T@eps_c@eps_c.T@X_c


            return (XX_inv@sigma_matrix@XX_inv)


    def standard_dev(self):
        return (np.sqrt(np.diagonal(self.AVAR()))).reshape((len(self.coefficients()),1))

    def t_stats(self):
        return (np.round(self.coefficients() / self.standard_dev(),2))

    def p_value(self):
        tvec = self.t_stats()
        dfree = len(self.prep_data().get('sub_sample').drop(self.dependent, axis = 1)) - 1
        return np.round(scipy.stats.norm.sf(abs(tvec)) * 2,2)

    def confidence(self):
        betas = self.coefficients()
        std = self.standard_dev()
        low = betas - std*1.96
        high = betas + std*1.96

        return {'low': low, 'high': high}

    def summary(self):
        header = [self.dependent[0], 'coefficient', 'se', 't', 'p_value', 'low 95', 'high 95']
        table = []
        vars = self.reggressors + self.cons_array
        def reshaping(array):
            return array[0:len(vars)].reshape((1,len(vars)))[0]

        vec = [vars, reshaping(self.coefficients()), reshaping(self.standard_dev()),
        reshaping(self.t_stats()),reshaping(self.p_value()), reshaping(self.confidence().get('low')),
        reshaping(self.confidence().get('high'))]
        vec = list(map(list, zip(*vec)))
        print('OLS Regression')

        print('------------------------------------------------------------------------------------')
        print(tabulate(vec, headers=header))
        print('------------------------------------------------------------------------------------')
        return ''


class two_sls():

    def __init__(self, dataset, dependent, regressors, endogenous, instruments,
                 cons = True, fixed_effects = []):
        self.dataset = dataset
        self.dependent = dependent
        self.regressors = regressors
        self.endogenous = endogenous
        self.instruments = instruments
        self. cons = cons
        if self.cons is True:
            self.cons_arr = ['cons']
        else:
            self.cons_arr = []
        self.fixed_effects = fixed_effects

    def retrive_data(self):
        data = self.dataset[self.dependent + self.regressors +
                            self.endogenous + self.instruments].dropna()
        if len(self.fixed_effects) == 0:
            return data
        else:
            dummies = pd.get_dummies(self.fixed_effects, drop_first= True)
            return pd.concat([data, dummies], axis = 1)


    def first_stage(self):
        # define list of regressors for first stage

        fs_regressors = [i for i in self.retrive_data().columns.tolist() if
                         i not in self.dependent + self.endogenous]

        #initializing the linear regression for the first stage
        betas_matrix = np.zeros((len(fs_regressors)+1
                                 , 1))
        fitted_matrix = np.zeros((len(self.retrive_data()),1))

        for end in range(len(self.endogenous)):
            fs_obj = ols(dataset=self.retrive_data(), regressors=fs_regressors, dependent=[self.endogenous[end]]
                         , fixed_effects=self.fixed_effects)

            betas_matrix= np.concatenate([betas_matrix, fs_obj.coefficients()], axis=1)
            fitted_matrix =  np.concatenate([fitted_matrix, fs_obj.fitted()], axis=1)


        betas_matrix = betas_matrix[:,1:]
        fitted_matrix = fitted_matrix[:, 1:]
        return {'betas' : betas_matrix, 'fitted' : fitted_matrix}


    def second_stage(self):
        # attach first stage fitted values in original datsaet
        first_dataset = self.retrive_data()
        for end in range(len(self.endogenous)):
            first_dataset[f'1stage_{self.endogenous[end]}'] = self.first_stage().get('fitted')[:,end]


        ss_xs = self.regressors + [i for i in first_dataset.columns.tolist() if
                                       i not in self.dependent + self.endogenous
                                       + self.instruments + self.regressors]


        # initialize model for 2nd stage regression
        model_ss = ols(first_dataset, dependent=self.dependent, regressors=ss_xs, fixed_effects=self.fixed_effects)
        # computing beta coefficients
        b_hat = model_ss.coefficients()
        Z = first_dataset[self.regressors + self.instruments + self.cons_arr]
        # get matrix with x variables
        sub_sample_2nd = ols(first_dataset, dependent=self.dependent, regressors=self.regressors + self.endogenous,
                             fixed_effects=self.fixed_effects).prep_data() \
            .get('sub_sample')
        X_2nd = sub_sample_2nd.drop(self.dependent, axis=1)
        Y = (sub_sample_2nd[self.dependent[0]].values)
        Y = Y.reshape((len(Y), 1))

        # computing fitted values for 2nd stage residuals
        XB_hat = (X_2nd @ b_hat).values
        XB_hat = XB_hat.reshape((len(XB_hat), 1))
        # computing 2nd stage residuals
        residual_2nd = Y - XB_hat
        # computing sigma 2nd stage
        sigma_hat = (residual_2nd.T @ residual_2nd) / (
                len(residual_2nd) - len(self.regressors + self.endogenous))
        # computing 2nd stage variance covariance matrix
        X_hat_second = (Z.T @ X_2nd).values
        X_hat_first = (np.linalg.inv(Z.T @ Z))
        X_hat = Z @ (X_hat_first @ X_hat_second)
        vars_matrix = sigma_hat * np.linalg.inv(X_hat.T @ X_hat)
        std_var = np.sqrt(np.diagonal(vars_matrix)).reshape((len(b_hat), 1))
        # computing t statistic
        t = np.round(model_ss.coefficients() / std_var, 2)
        # computing p value
        dfree = len(model_ss.prep_data().get('sub_sample')) - 1
        p_val = np.round(scipy.stats.t.sf(abs(t), df=dfree) * 2, 3)

        # computing confidence bands
        low = model_ss.coefficients() - (std_var * 1.96)
        high = model_ss.coefficients() + (std_var * 1.96)

        return {'beta': model_ss.coefficients(), 'std': std_var, 't': t, 'p': p_val, 'low': low, 'high': high,
                'var_matrix': vars_matrix}

    def summary(self):
        print('2SLS Regression')
        header = [self.dependent, 'coefficient', 'se', 't', 'p_value', 'low 95', 'high 95']
        table = []
        vars = self.regressors + self.endogenous + self.cons_arr

        vec = [vars, self.second_stage().get('beta'), self.second_stage().get('std'), self.second_stage().get('t'),
        self.second_stage().get('p'), self.second_stage().get('low'),  self.second_stage().get('high')]
        vec = list(map(list, zip(*vec)))

        print('-------------------------------------------------------------------------------')
        print(tabulate(vec, headers=header))
        print('-------------------------------------------------------------------------------')
        print(f'Endogenous variable: {self.endogenous}')
        print(f'Instruments: {self.regressors + self.instruments} ')
        print('-------------------------------------------------------------------------------')


        return ''


# EXRECISE 5.4
references: Wooldridge, J. M. (2010). Econometric Analysis of Cross Section and Panel Data. The MIT Press. http://www.jstor.org/stable/j.ctt5hhcfr


In [6]:
# LOAD CARD DATASET (Wooldridge)
import pandas as pd
data = pd.read_stata('card.dta') 

data

,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,...,smsa66,wage,enroll,kww,iq,married,libcrd14,exper,lwage,expersq
0,2,0,0,7,29,NaN,NaN,158413.0,1,0,...,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,3,0,0,12,27,8.0,8.0,380166.0,1,0,...,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,4,0,0,12,34,14.0,12.0,367470.0,1,0,...,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,5,1,1,11,27,11.0,12.0,380166.0,1,0,...,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,6,1,1,12,34,8.0,7.0,367470.0,1,0,...,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,5218,0,1,12,25,8.0,12.0,82135.0,1,0,...,0,335,0,15.0,NaN,1.0,0.0,7,5.814130,49
3006,5219,0,1,13,34,NaN,NaN,88765.0,1,0,...,0,481,0,43.0,NaN,1.0,1.0,15,6.175867,225
3007,5220,0,1,12,24,11.0,NaN,89271.0,0,0,...,0,500,0,25.0,109.0,1.0,0.0,6,6.214608,36
3008,5221,0,1,12,31,NaN,NaN,110376.0,1,0,...,0,713,0,32.0,107.0,1.0,1.0,13,6.569481,169


In [7]:
# perform the ols regression

ols_reg = ols(dataset = data, dependent=['lwage'], regressors=['educ','exper', 'expersq', 'black' ,'south', 'smsa', 'reg661'
                                                              ,'reg662','reg663','reg664','reg665'
                                                              ,'reg666','reg667','reg668','smsa66'])
ols_reg.summary()

OLS Regression
------------------------------------------------------------------------------------
lwage      coefficient           se       t    p_value       low 95      high 95
-------  -------------  -----------  ------  ---------  -----------  -----------
educ       0.0746933    0.00349835    21.35       0      0.0678365    0.08155
exper      0.084832     0.00662422    12.81       0      0.0718486    0.0978155
expersq   -0.00228704   0.000316626   -7.22       0     -0.00290763  -0.00166645
black     -0.199012     0.0182483    -10.91       0     -0.234779    -0.163246
south     -0.147955     0.0259799     -5.69       0     -0.198876    -0.0970345
smsa       0.136385     0.0201005      6.79       0      0.0969876    0.175781
reg661    -0.11857      0.0388301     -3.05       0     -0.194677    -0.0424628
reg662    -0.0222026    0.0282575     -0.79       0.43  -0.0775874    0.0331822
reg663     0.0259703    0.0273644      0.95       0.34  -0.0276639    0.0796044
reg664    -0.0634942 

''

In [8]:
# perform 2sls regression

twosls_reg = two_sls(dataset = data, dependent=['lwage'], regressors=['exper', 'expersq', 'black' ,'south', 'smsa', 'reg661'
                                                              ,'reg662','reg663','reg664','reg665'
                                                              ,'reg666','reg667','reg668','smsa66'],endogenous=['educ'],instruments=['nearc4'] 
                    )

twosls_reg.summary()

2SLS Regression
-------------------------------------------------------------------------------
['lwage']      coefficient           se      t    p_value       low 95      high 95
-----------  -------------  -----------  -----  ---------  -----------  -----------
exper           0.108271    0.0236546     4.58      0       0.061908     0.154634
expersq        -0.00233494  0.000333441  -7         0      -0.00298848  -0.00168139
black          -0.146776    0.0538909    -2.72      0.007  -0.252402    -0.0411497
south          -0.144672    0.0272801    -5.3       0      -0.19814     -0.0912026
smsa            0.111808    0.0316567     3.53      0       0.0497612    0.173855
reg661         -0.107814    0.0418067    -2.58      0.01   -0.189755    -0.0258731
reg662         -0.00704645  0.0329018    -0.21      0.834  -0.0715339    0.057441
reg663          0.0404445   0.0317753     1.27      0.204  -0.021835     0.102724
reg664         -0.0579172   0.0375996    -1.54      0.124  -0.131612     0.

''

# Exercise 5.8

In [11]:
nls80 = pd.read_stata('nls80.dta')

In [13]:
twosls_reg = two_sls(dataset = nls80, dependent=['lwage'], regressors=['exper', 'tenure', 'married' ,'south', 'urban', 'black']
                                                              ,endogenous=['educ', 'iq'],instruments=['kww','meduc', 'feduc', 'sibs'] 
                    )

twosls_reg.summary()

2SLS Regression
-------------------------------------------------------------------------------
['lwage']      coefficient          se      t    p_value        low 95     high 95
-----------  -------------  ----------  -----  ---------  ------------  ----------
exper           0.0313987   0.0122452    2.56      0.011   0.00739819   0.0553992
tenure          0.00704757  0.00336934   2.09      0.037   0.000443671  0.0136515
married         0.213337    0.053491     3.99      0       0.108494     0.318179
south          -0.0941667   0.0506034   -1.86      0.063  -0.193349     0.00501605
urban           0.168072    0.0384068    4.38      0       0.0927947    0.243349
black          -0.234571    0.224599    -1.04      0.299  -0.674786     0.205643
educ            0.16469     0.113187     1.46      0.145  -0.0571553    0.386536
iq             -0.0102736   0.0199983   -0.51      0.61   -0.0494704    0.0289231
cons            4.93296     0.486671    10.14      0       3.97909      5.88684
-----

''